In [1]:
TMVA::Tools::Instance();
TString outfilename = "higgs_dnn_256.root";
TFile *output       = TFile::Open(outfilename, "RECREATE");
TMVA::Factory *factory   = new TMVA::Factory("TMVAClassification", output,
                                             "AnalysisType=Classification");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 
(TMVA::Factory *) 0x7f1c903587b0


## Load Data

In [2]:
TFile *input              = TFile::Open("higgs.root");
TMVA::DataLoader *loader  = new TMVA::DataLoader("higgs");
TTree *tsignal     = (TTree *) input->Get("TreeS");
TTree *tbackground = (TTree *) input->Get("TreeB");
loader->AddSignalTree(tsignal, 1.0);
loader->AddBackgroundTree(tbackground, 1.0);

loader->AddVariable("lepton_pT",'F');
loader->AddVariable("lepton_eta",'F');
loader->AddVariable("lepton_phi",'F');
loader->AddVariable("missing_energy_magnitude",'F');
loader->AddVariable("missing_energy_phi",'F');
loader->AddVariable("jet_1_pt",'F');
loader->AddVariable("jet_1_eta",'F');
loader->AddVariable("jet_1_phi",'F');
loader->AddVariable("jet_1_b_tag",'F');
loader->AddVariable("jet_2_pt",'F');
loader->AddVariable("jet_2_eta",'F');
loader->AddVariable("jet_2_phi",'F');
loader->AddVariable("jet_2_b_tag",'F');
loader->AddVariable("jet_3_pt",'F');
loader->AddVariable("jet_3_eta",'F');
loader->AddVariable("jet_3_phi",'F');
loader->AddVariable("jet_3_b_tag",'F');
loader->AddVariable("jet_4_pt",'F');
loader->AddVariable("jet_4_eta",'F');
loader->AddVariable("jet_4_phi",'F');
loader->AddVariable("jet_4_b_tag",'F');
loader->AddVariable("m_jj",'F');
loader->AddVariable("m_jjj",'F');
loader->AddVariable("m_lv",'F');
loader->AddVariable("m_jlv",'F');
loader->AddVariable("m_bb",'F');
loader->AddVariable("m_wbb",'F');
loader->AddVariable("m_wwbb",'F');

--- DataSetInfo              : Dataset[higgs] : Added class "Signal"	 with internal class number 0
--- higgs                    : Add Tree TreeS of type Signal with 5829123 events
--- DataSetInfo              : Dataset[higgs] : Added class "Background"	 with internal class number 1
--- higgs                    : Add Tree TreeB of type Background with 5170877 events


In [3]:
TString dataString = "nTrain_Signal=5000000:"
                     "nTrain_Background=5000000:"
                     "nTest_Signal=100000:"
                     "nTest_Background=100000:"
                     "SplitMode=Random:"
                     "NormMode=NumEvents:"
                     "!V";
loader->PrepareTrainingAndTestTree("", "", dataString);
loader->PrepareTrainingAndTestTree("", "", dataString);

--- higgs                    : Preparing trees for training and testing...
--- higgs                    : Preparing trees for training and testing...


## Network Structure and Training Strategy

In [4]:
TString configString = "!H:V";
configString += ":Architecture=GPU";
configString += ":VarTransform=N";
configString += ":ErrorStrategy=CROSSENTROPY";
configString += ":WeightInitialization=XAVIER";
TString layoutString    = "Layout=TANH|256,TANH|256,TANH|256,TANH|256,TANH|256,LINEAR";
TString trainingString1 = "TrainingStrategy="
                         "LearningRate=0.1,"
                         "ConvergenceSteps=10,"
                         "Momentum=0.9,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-6,"
                         "TestRepetitions=5";
TString trainingString2 = "LearningRate=0.01,"
                         "ConvergenceSteps=10,"
                         "Momentum=0.9,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-6,"
                         "TestRepetitions=5";
TString trainingString3 = "LearningRate=0.001,"
                         "ConvergenceSteps=10,"
                         "BatchSize=4096,"
                         "Momentum=0.9,"
                         "Regularization=L2,"
                         "WeightDecay=1e-6,"
                         "TestRepetitions=5";
TString trainingString4 = "LearningRate=0.0001,"
                         "ConvergenceSteps=10,"
                         "Momentum=0.5,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-6,"
                         "TestRepetitions=5";
TString trainingString5 = "LearningRate=0.00001,"
                         "ConvergenceSteps=10,"
                         "Momentum=0.0,"
                         "BatchSize=4096,"
                         "TestRepetitions=5"
                         "Regularization=L2,"
                         "WeightDecay=1e-6";

configString += ":" + trainingString1 + "|" + trainingString2 + "|" + trainingString3;
configString += "|" + trainingString4 + "|" + trainingString5;
configString += ":" + layoutString;
factory->BookMethod(loader,  TMVA::Types::kDNN, "DNN", configString);

--- Factory                  : Booking method: DNN DataSet Name: higgs
--- <VERBOSE> Configurable             : Parsing option string: 
--- <VERBOSE> Configurable             : ... "!H:V:Architecture=GPU:VarTransform=N:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIER:TrainingStrategy=LearningRate=0.1,ConvergenceSteps=10,Momentum=0.9,BatchSize=4096,Regularization=L2,WeightDecay=1e-6,TestRepetitions=5|LearningRate=0.01,ConvergenceSteps=10,Momentum=0.9,BatchSize=4096,Regularization=L2,WeightDecay=1e-6,TestRepetitions=5|LearningRate=0.001,ConvergenceSteps=10,BatchSize=4096,Momentum=0.9,Regularization=L2,WeightDecay=1e-6,TestRepetitions=5|LearningRate=0.0001,ConvergenceSteps=10,Momentum=0.5,BatchSize=4096,Regularization=L2,WeightDecay=1e-6,TestRepetitions=5|LearningRate=0.00001,ConvergenceSteps=10,Momentum=0.0,BatchSize=4096,TestRepetitions=5Regularization=L2,WeightDecay=1e-6:Layout=TANH|256,TANH|256,TANH|256,TANH|256,TANH|256,LINEAR"
--- <VERBOSE> Configurable             : The follo

## Train and Evaluate Method

In [5]:
factory->TrainAllMethods();
factory->TestAllMethods();
factory->EvaluateAllMethods();

--- Factory                  :  
--- Factory                  : Train all methods for Classification ...
--- DataSetFactory           : Dataset[higgs] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[higgs] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[higgs] :     Signal          -- number of events       : 5829123  / sum of weights: 5.82912e+06
--- DataSetFactory           : Dataset[higgs] :     Background      -- number of events       : 5170877  / sum of we

--- DNN                      : Dataset[higgs] : Elapsed time for evaluation of 10000000 events: 0.195 sec       


--- DNN                      : Dataset[higgs] : Creating weight file in xml format: higgs/weights/TMVAClassification_DNN.weights.xml
--- DNN                      : Dataset[higgs] : Creating standalone response class: higgs/weights/TMVAClassification_DNN.class.C
--- Factory                  : Training finished
--- Factory                  : 
--- Factory                  : Ranking input variables (method specific)...
--- DNN                      : Ranking result (top variable is best ranked)
--- DNN                      : -------------------------------------------------
--- DNN                      : Rank : Variable                 : Importance
--- DNN                      : -------------------------------------------------
--- DNN                      :    1 : lepton_pT                : 1.000e+00
--- DNN                      :    2 : lepton_eta               : 1.000e+00
--- DNN                      :    3 : lepton_phi               : 1.000e+00
--- DNN                      :    4 : miss

--- DNN                      : Dataset[higgs] : Elapsed time for evaluation of 200000 events: 880 sec       
--- Factory                  : Evaluate all methods...
--- Factory                  : Evaluate classifier: DNN
--- TFHandler_DNN            : -------------------------------------------------------------------------------------------------------------------------------------------
--- TFHandler_DNN            :                 Variable                        Mean                        RMS                [        Min                        Max ]
--- TFHandler_DNN            : -------------------------------------------------------------------------------------------------------------------------------------------
--- TFHandler_DNN            :                lepton_pT:                  -0.87845                  0.096288   [                   -1.0000                   0.22944 ]
--- TFHandler_DNN            :               lepton_eta:                -0.0011346                   0.

## Plot Results

%jsroot on
TCanvas *a=factory->GetROCCurve(loaderHighLevel);
a->Draw();